# Prototype of reaction-diffusion on temporal networks

Install the packages

In [1]:
Pkg.update()
Pkg.add("Distributions")
Pkg.add("LightGraphs")
Pkg.add("SimJulia")
Pkg.checkout("SimJulia")

INFO: Updating METADATA...
INFO: Updating SimJulia master...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package Distributions is already installed
INFO: Package LightGraphs is already installed
INFO: Package SimJulia is already installed
INFO: Checking out SimJulia master...
INFO: Pulling SimJulia latest master...
INFO: No packages to install, update or remove


Making packages available in notebook

In [2]:
using Distributions
using LightGraphs
using ResumableFunctions
using SimJulia

Define a type to hold the state of the simulation

In [3]:
mutable struct SimulationState
    network :: DiGraph
    link_up :: Event
end

Define the temporal behaviour of the network

In [8]:
@resumable function edge_behaviour(sim::Simulation, next_state::Bool, src::Int, dst::Int, sim_state::SimulationState)
    up_distribution = Exponential(1.0) # other distributions are available
    down_distribution = Exponential(1.0) # other distributions are available
    while true
        if next_state
            println(now(sim), ": Edge from ", src, " to ", dst, " up")
            add_edge!(sim_state.network, src, dst)
            next_state = false
            succeed(sim_state.link_up, value=src)
            sim_state.link_up = Event(sim)
            @yield Timeout(sim, rand(up_distribution))
        else
            println(now(sim), ": Edge from ", src, " to ", dst, " down")
            rem_edge!(sim_state.network, src, dst)
            next_state = true
            @yield Timeout(sim, rand(down_distribution))
        end
    end
end

edge_behaviour (generic function with 2 methods)

Define the walker behaviour

In [5]:
@resumable function walker_behaviour(sim::Simulation, walker::Int, vertex::Int, sim_state::SimulationState)
    wait_distribution = Exponential(1.0) # other distributions are available
    while true
        println(now(sim), ": Walker ", walker, " arrived in vertex ", vertex)
        @yield Timeout(sim, rand(wait_distribution))
        connections = out_neighbors(sim_state.network, vertex)
        println(now(sim), ": Walker ", walker, " in vertex ", vertex, " has connections to ", connections)
        #isempty(connections) && continue # no connections available, so we wait another period
        if isempty(connections)
            src = 0
            while src != vertex
                println(now(sim), ": Walker ", walker, " in vertex ", vertex, " is waiting for connections")
                src = @yield sim_state.link_up
            end
        end
        connection_distribution = DiscreteUniform(1, length(connections))
        vertex = connections[Int(rand(connection_distribution))]
    end
end

walker_behaviour (generic function with 1 method)

Define the initial simulation settings

In [6]:
function play(run_time::Float64, nr_vertices::Int, edge_probability::Float64, nr_walkers::Int)
    sim = Simulation()
    network = erdos_renyi(nr_vertices, edge_probability, is_directed=true) # modify parameter to change edge probability
    sim_state = SimulationState(DiGraph(nr_vertices, ne(network)), Event(sim)) 
    edge_distribution = Bernoulli(0.5) # modify parameter to change the density of initial up edges
    walker_distribution = DiscreteUniform(1, nr_vertices) 
    for edge in edges(network) 
        state = Bool(rand(edge_distribution))
        @coroutine edge_behaviour(sim, state, src(edge), dst(edge), sim_state)
    end
    for walker in 1:nr_walkers
        vertex = Int(rand(walker_distribution))
        @coroutine walker_behaviour(sim, walker, vertex, sim_state)
    end
    run(sim, run_time)
end

play (generic function with 1 method)

In [9]:
play(5.0, 10, 0.25, 10)

0.0: Edge from 1 to 4 up
0.0: Edge from 1 to 9 up
0.0: Edge from 2 to 7 up
0.0: Edge from 5 to 8 down
0.0: Edge from 6 to 2 up
0.0: Edge from 6 to 10 up
0.0: Edge from 7 to 2 down
0.0: Edge from 7 to 5 down
0.0: Edge from 7 to 10 up
0.0: Edge from 8 to 1 down
0.0: Edge from 8 to 6 down
0.0: Edge from 9 to 1 up
0.0: Edge from 10 to 1 up
0.0: Edge from 10 to 7 up
0.0: Edge from 10 to 9 up
0.0: Walker 1 arrived in vertex 8
0.0: Walker 2 arrived in vertex 3
0.0: Walker 3 arrived in vertex 9
0.0: Walker 4 arrived in vertex 9
0.0: Walker 5 arrived in vertex 7
0.0: Walker 6 arrived in vertex 8
0.0: Walker 7 arrived in vertex 6
0.0: Walker 8 arrived in vertex 10
0.0: Walker 9 arrived in vertex 10
0.0: Walker 10 arrived in vertex 9
0.04539107431248901: Walker 5 in vertex 7 has connections to [4, 8, 10]
0.04539107431248901: Walker 5 arrived in vertex 10
0.11508724726939576: Edge from 1 to 4 down
0.11946260131464309: Edge from 10 to 7 down
0.12751829910106355: Walker 8 in vertex 10 has connection